<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/Rest_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv

In [4]:
file = open('combine.csv')
type(file)
csvreader = csv.reader(file)

In [ ]:
header = []
header = next(csvreader)
header